# Extractor de features para estudio de células de melanopsina
**Importante:** se asume que el usuario tiene instalada la biblioteca *scipy* a través de su instalador o *Anaconda*. Además es necesario tener instaladas las librerías *spikelib* y *tqdm*, que puede ser descargada rápidamente descomentando el comando de la siguiente celda.

In [ ]:
#%pip install spikelib tqdm

In [ ]:
from configparser import ConfigParser, ExtendedInterpolation

import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import gc

from ipywidgets import interact, interact_manual, IntSlider, Dropdown, fixed
from spikelib.utils import check_directory
from sorting import get_features, plot_raster
from scipy.io import savemat, loadmat
from tqdm import tqdm

%matplotlib inline

## Adquisición de datos

In [ ]:
exp_name = 'MR-261_t2'
sorting_file = 'experiments/{}/{}.result.hdf5'.format(exp_name, exp_name)

isi_bin = 2
bins = np.linspace(0, isi_bin * 100, 101)

## Intervalos
Intervalos de los tiempos (en segundos) de los estímulos utilizados en el experimento. Pueden ser descritos a mano o importados desde un archivo csv en caso de existir, donde las columnas deben tener por nombre *start_event* y *end_event*, mientras que los índices corresponden al nombre del estímulo a elección del usuario.

In [ ]:
int_file = 'experiments/{}/{}-intervals.csv'.format(exp_name, exp_name)
intervals = {}
try:
    df = pd.read_csv(int_file, index_col=0)
    for idx in df.index:
        intervals[idx] = tuple(df.loc[idx])
    intervals['total'] = (df['start_event'][0], df['end_event'][-1])
    
except:
    intervals['wn'] = (2.390, 1207.4)
    intervals['spont'] = (1209.4, 1521.2)
    intervals['chirp'] = (1526.8, 2188.4)
    intervals['mchirp'] = (2359.323, 3629.5)
    intervals['bflash'] = (3622.3, 3939.3)
    intervals['gflash'] = (3940.2, 4236.6)
    intervals['rflash'] = (4293.2, 4568.1)

    # Drugs
    intervals['d_chirp'] = (4641.5, 5342.1)
    intervals['d_mchirp'] = (5346.1, 6774.7)
    intervals['d_bflash'] = (6760.1, 7080.5)
    intervals['d_gflash'] = (7079.8, 7419.5)
    intervals['d_rflash'] = (7469.8, 7739.5)
            
start_point = np.array([v[0] for v in intervals.values()]) * 20000.0

## Selección de células
Se define la ruta al archivo con los índices numéricos de las células que se espera sean exploradas. Si la ruta está mal descrita, el nombre del archivo está erróneo o simplemente el archivo no existe se tomarán todas las células del archivo sorting.

**Warning:** Se considera que el archivo posee índices desde i >= 1 y por tanto se les resta a todos -1 debido a que las células de los archivos hdf5 poseen índices i >= 0.

In [ ]:
# Modificar nombre del archivo
index_file = 'experiments/{}/cell_index.txt'.format(exp_name)
try:
    file_index = open(index_file, 'r') 
    lines = file_index.readlines() 
    cell_index = [int(idx) - 1 for idx in lines]
    cell_keys = ['temp_{}'.format(int(idx) - 1) for idx in lines]
except FileNotFoundError:
    with h5py.File(sorting_file, 'r') as f:
        cell_keys = list(f['/spiketimes'].keys())

In [ ]:
with h5py.File(sorting_file, 'r') as f:
    features, isi = get_features(f)
    print(features.shape)

## Preparación de directorios de guardado

In [ ]:
def check_exp_dir(exp):
    exp_dir = 'results/{}'.format(exp)
    if os.path.isdir('results') == False:
        os.mkdir('results')
    if os.path.isdir(exp_dir) == False:
        os.mkdir(exp_dir)        
    return exp_dir

def check_isi_dir(exp, cell_key):
    exp_dir = check_exp_dir(exp)
    isi_dir = os.path.join(exp_dir, 'isi')
    
    if os.path.isdir(isi_dir) == False:
        os.mkdir(isi_dir)
    return isi_dir    

def check_autocorr_dir(exp, cell_key):
    exp_dir = check_exp_dir(exp)
    ac_dir = os.path.join(exp_dir, 'autocorrelograms')
    cell_ac_dir = os.path.join(ac_dir, cell_key)
    
    if os.path.isdir(ac_dir) == False:
        os.mkdir(ac_dir)
    if os.path.isdir(cell_ac_dir) == False:
        os.mkdir(cell_ac_dir)        
    return cell_ac_dir

# Figuras
## Histogramas ISI y firingrate
Función que genera los gráficos de ISI para visualización y almacenamiento en disco. El objecto *fig* se entrega como argumento para evitar memory leaks durante iteraciones.

In [ ]:
def plot_isi(cell_key, fig, avg_sr=False):
    if fig == None:
        fig = plt.figure()
    with h5py.File(sorting_file, 'r') as sorting:
        spks = sorting['/spiketimes'][cell_key][...].flatten() / 20000.0
        dur = sorting['/info/duration/'][...]
        spike_rate = (spks.size / dur.astype(float)).flatten()[0]
        
        # Computing isi
        dff = np.diff(spks) if spks.any() else 0
        count, bins = np.histogram(dff, bins=np.arange(0, 1, 0.01))
        
        # Print spike_rate
        if avg_sr == True:
            print('Average spike rate: {} [spks/s]'.format(spike_rate))
    
    ax = fig.add_subplot()
    ax.set_xticks([0, 0.25, 0.5, 0.75, 1])
    ax.set_xticklabels([0, 250, 500, 750, 1000])
    ax.set_xlabel('Time [ms]')
    ax.hist(dff, bins=np.arange(0, 1, 0.01))
    ax.set_title(cell_key)    

## Visualizador de ISI por célula

In [ ]:
interact(plot_isi,
         cell_key=Dropdown(options=cell_keys),
         fig=fixed(None),
         avg_sr=fixed(True));

## Guardar figuras de ISI
Esta celda de código almacena en el directorio *results* las imágenes de ISI por célula.

In [ ]:
fig = plt.figure()
for cell in tqdm(cell_keys):
    plot_isi(cell, fig)
    save_file = os.path.join(check_isi_dir(exp_name, cell_key),
                             cell_key)
    fig.savefig(save_file)
    fig.clear()
    fig.clf()
    plt.close(fig)

In [ ]:
fig, ax = plt.subplots(2)
ax[0].hist(features[:, 0], bins=100)
ax[1].hist(features[:, 1], bins=100)
ax[0].set(title='Firingrate', xlabel='firing rate [spk/s]', ylabel='frequency')
ax[1].set(title='ISI', xlabel='bins [samples]', ylabel='frequency')
fig.tight_layout()
fig.savefig(os.path.join(check_exp_dir(exp_name),
                         'isi_fr_histogram.png'))

## Spike rates
Función para obtener *avg spike rate* de una célula a lo largo del experimento para los distintos tipos de estímulos, que retorna un arreglo con los distintos valores obtenidos. Posee una opción para printear los resultados y que omite el retorno de variables.

In [ ]:
def get_spike_rates(cell_key, printable=False):
    with h5py.File(sorting_file, 'r') as sorting:
        spike_rates = {}
        spks = sorting['/spiketimes'][cell_key][...].flatten() / 20000.0
        dur = sorting['/info/duration/'][...]
        
        for key, val in intervals.items():
            spikes = spks[(spks >= intervals[key][0])*(spks < intervals[key][1])]
            spike_rates[key] = spikes.size / np.diff(intervals[key])[0]
            
            if printable == True:
                print('{} spike rate: {:.4f} [spks/s]'.format(key, spike_rates[key])) 

        if printable == False:
            data = np.array(list(spike_rates.values()))
            return data

## Visualizador de spike rates

In [ ]:
interact(get_spike_rates,
         cell_key=Dropdown(options=cell_keys),
         printable=fixed(True));

## Guardar datos de spike rates de todas las células

In [ ]:
save_file = os.path.join(check_exp_dir(exp_name),
                         'spike_rates.csv')

data = np.zeros((len(cell_keys), len(intervals.items())))
for idx, cell in enumerate(tqdm(cell_keys)):
    data[idx, :] = get_spike_rates(cell, False)
    
df = pd.DataFrame(data, index=cell_keys, columns=intervals.keys())
df.to_csv(save_file, mode='w')
gc.collect();

# Correlogramas
Se definen las funciones para obtener autocorrelogramas y crosscorrelogramas.
## Autocorrelogramas
Función para obtener y graficar autocorrelograma. Es necesario indicar el "nombre" de la célula y sección del experimento.

In [ ]:
def get_autocorrelogram(cell_key, interval, width, lim):
    num_bins = int(lim / width)
        
    with h5py.File(sorting_file, 'r') as sorting:
        spks = sorting['/spiketimes'][cell_key][...].flatten() / 20000.0
        
        spikes = spks[(spks >= interval[0])*(spks < interval[1])]
        
        if spikes.shape[0] > 0:
            # Auto-correlation
            corr = np.zeros(num_bins)
            for idx, _ in enumerate(spikes):
                d = spikes - spikes[idx]
                h, bins = np.histogram(d, bins=num_bins, range=(-lim, lim))
                corr = corr + h
            
            corr /= np.diff(interval)
            
            # Normalización
            #corr -= spikes.shape[0] ** 2 * width / (np.diff(intervals[int_key]) ** 2)
            
            # Bin 0 seteado a cero -> avg spike rate
            corr[np.argmax(corr)] = 0
        else:
            corr = np.nan * np.empty(num_bins)
            bins = np.nan * np.empty(num_bins + 1) # por cómo es retornado
        
        return bins[1:], corr
    
def plot_autocorrelogram(cell_key, interval, width, lim, fig):
    bins, corr = get_autocorrelogram(cell_key, interval, width, lim)
    
    if fig == None:
        fig = plt.figure()
        
    ax = fig.add_subplot()
    ax.set_xticks([-lim, -lim * 0.5, 0, lim * 0.5, lim])
    ax.set_xticklabels(np.array([-lim, -lim * 0.5, 0, lim * 0.5, lim]) * 1000)
    ax.set_xlabel('Time [ms]')
    ax.bar(bins, corr, width * 2)

## Visualizador de autocorrelogramas

In [ ]:
# Se definen el ancho de los bins y el límite del histograma en [s]
width = 0.001
lim = 0.2

interact(plot_autocorrelogram,
         cell_key=Dropdown(options=cell_keys),
         interval=Dropdown(options=intervals.items()),
         width=fixed(width),
         lim=fixed(lim),
         fig=fixed(None));

## Guardar figuras de autocorrelogramas

La siguiente celda itera por los identificadores de las células definidas por *cell_keys* en una celda de código anterior. Es necesario tener los intervalos de tiempo definidos en el diccionario *intervals*.

In [ ]:
# Se definen el ancho de los bins y el límite del histograma en [s]
width = 0.001
lim = 0.2

# Crea directorio results y almacena las figuras de autocorrelogramas para cada célula del exp
# y por intervalos de los tiempos de cada estímulo
fig = plt.figure()

for cell_key in tqdm(cell_keys):
    for key, interval in intervals.items():        
        plot_autocorrelogram(cell_key, interval, width, lim, fig)
    
        fig.savefig(os.path.join(check_autocorr_dir(exp_name, cell_key),
                                 '{}.png'.format(key)))
        fig.clf()
        fig.clear()
        plt.cla()
        plt.close(fig)

## Guardar autocorrelogramas en hdf5

La siguiente celda realiza el cómputo de autocorrelogramas y los almacena en un archivo *hdf5* durante su ejecución. Esto evita que los cálculos sean almacenados en la memoria RAM y se guarden directamente en un archivo en el disco duro.

In [ ]:
# Se definen el ancho de los bins y el límite del histograma en [s]
width = 0.001
lim = 0.2

output = os.path.join(check_exp_dir(exp_name), 'autocorrelogram.hdf5')
for cell_key in tqdm(cell_keys):
    for key, interval in intervals.items():
        bins, corr = get_autocorrelogram(cell_key, interval, width, lim)
        with h5py.File(output, 'a') as autocorr:
            cell_grp = autocorr.require_group(cell_key)
            if 'bins' not in cell_grp:
                cell_grp.create_dataset('bins', bins.shape, data=bins)
            else:
                cell_grp['bins'][:] = bins
                
            if key not in cell_grp:
                cell_grp.create_dataset(key, corr.shape, data=corr)
            else:
                cell_grp[key][:] = corr

## Crosscorrelogramas

In [ ]:
def get_crosscorrelogram(cell0_key, cell1_key, interval, width, lim):
    num_bins = int(lim / width)
        
    with h5py.File(sorting_file, 'r') as sorting:
        spks0 = sorting['/spiketimes'][cell0_key][...].flatten() / 20000.0
        spks1 = sorting['/spiketimes'][cell1_key][...].flatten() / 20000.0
        
        spikes0 = spks0[(spks0 >= interval[0])*(spks0 < interval[1])]
        spikes1 = spks1[(spks1 >= interval[0])*(spks1 < interval[1])]
        
        if spikes0.shape[0] > 0 and spikes1.shape[0] > 0:
            # Cross-correlation
            corr = np.zeros(num_bins)
            for idx, _ in enumerate(tqdm(spikes1)):
                d = spikes0 - spikes1[idx]
                h, bins = np.histogram(d, bins=num_bins, range=(-lim, lim))
                corr = corr + h
            
            corr /= np.diff(interval)
                        
            # Bin 0 seteado a cero -> avg spike rate
            if cell0_key == cell1_key:
                corr[np.argmax(corr)] = 0
        else:
            corr = np.nan * np.empty(num_bins)
            bins = np.nan * np.empty(num_bins + 1) # por cómo es retornado
        
        return bins[1:], corr
    
def plot_crosscorrelogram(cell0_key, cell1_key, interval, width, lim, fig):
    bins, corr = get_crosscorrelogram(cell0_key, cell1_key, interval, width, lim)
    
    if fig == None:
        fig = plt.figure()
    
    ax = fig.add_subplot()
    ax.set_xticks([-lim, -lim * 0.5, 0, lim * 0.5, lim])
    ax.set_xticklabels(np.array([-lim, -lim * 0.5, 0, lim * 0.5, lim]) * 1000)
    ax.set_xlabel('Time [ms]')
    ax.bar(bins, corr, width * 2)
    
    del bins, corr
    
    return fig, ax

## Visualizador de crosscorrelograma

In [ ]:
interact(plot_crosscorrelogram,
         cell0_key=Dropdown(options=cell_keys),
         cell1_key=Dropdown(options=cell_keys),
         interval=Dropdown(options=intervals.items()),
         width=fixed(width),
         lim=fixed(lim),
         fig=fixed(None));

## ISI clustermap

In [ ]:
isi_max = isi.max(axis=1, )
isi_max[isi_max==0] = 1

save_file = os.path.join(check_exp_dir(exp_name),
                         'clustermap.png')

g = sns.clustermap(isi/isi_max[:, None], col_cluster=False, figsize=(10, 10))
g.savefig(save_file)

## Raster plot

Se genera Raster plot y se almacena en el directorio './results'.

Si se definió *cell_index* en celdas de código anteriores se incluirán en el raster solo aquellos índices que se encuentran en esta variable, en caso contrario se recomienda modificar *range_view* para el rango de las células que se desean graficar al comienzo de la siguiente celda

In [ ]:
# Si no existe cell_index, se define un rango de células para
# incluir en el raster
range_view = [0, 40]

try:
    cell_index
    if cell_index != None:
        range_view = None
except:
    cell_index = None
    
save_file = os.path.join(check_exp_dir(exp_name),
                         'rasterplot.png')
save_pdf = os.path.join(check_exp_dir(exp_name),
                        'rasterplot.pdf')

with h5py.File(sorting_file, 'r') as f:
    fig, ax = plot_raster(
        sorting=f,
        range_view=range_view,
        idx_units=cell_index,
        protocols_points=start_point,
        figsize=(10, 10),
    )
    ax.set(title='Raster {}'.format(exp_name))
    fig.savefig(save_file)
    fig.savefig(save_pdf)